这个笔记本看，怎么计算suffix 新增的参数。

In [6]:
!ls /data/MODELS/gpt2

config.json  merges.txt  pytorch_model.bin  tokenizer.json  vocab.json


## 回顾Peft Prefix Tuning

In [7]:
from peft import PrefixTuningConfig, get_peft_model, TaskType
from peft.peft_model import PeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    num_virtual_tokens=20
)
model = get_peft_model(
    model = AutoModelForCausalLM.from_pretrained("/data/MODELS/gpt2"),
    peft_config=peft_config,
)
print(model)


PeftModelForCausalLM(
  (base_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
  (prompt_encoder): Mo

```
(prompt_encoder): ModuleDict(
    (default): PrefixEncoder(
        (embedding): Embedding(20, 18432)
    )
)
```

这个模块是在`PeftModel._setup_prompt_encoder`里面加的

Line 392
```python
if config.peft_type == PeftType.PROMPT_TUNING:
    prompt_encoder = PromptEmbedding(config, self.word_embeddings)
elif config.peft_type == PeftType.MULTITASK_PROMPT_TUNING:
    prompt_encoder = MultitaskPromptEmbedding(config, self.word_embeddings)
elif config.peft_type == PeftType.P_TUNING:
    prompt_encoder = PromptEncoder(config)
elif config.peft_type == PeftType.PREFIX_TUNING:
    prompt_encoder = PrefixEncoder(config)
else:
    raise ValueError("Not supported")
```

---
### Prefix Encoder For PrefixTuning

```python
def __init__(self, config):
    super().__init__()
    self.prefix_projection = config.prefix_projection
    token_dim = config.token_dim
    num_layers = config.num_layers
    encoder_hidden_size = config.encoder_hidden_size
    num_virtual_tokens = config.num_virtual_tokens
    if self.prefix_projection and not config.inference_mode:
        # Use a two-layer MLP to encode the prefix
        self.embedding = torch.nn.Embedding(num_virtual_tokens, token_dim)
        self.transform = torch.nn.Sequential(
            torch.nn.Linear(token_dim, encoder_hidden_size),
            torch.nn.Tanh(),
            torch.nn.Linear(encoder_hidden_size, num_layers * 2 * token_dim),
        )
    else:
        self.embedding = torch.nn.Embedding(num_virtual_tokens, num_layers * 2 * token_dim)
```
默认如果没有prefix_projection的话，就是只有Embedding模式；

所以我们只需要 重载原有的forward就行了！

In [1]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/data/MODELS/gpt2")
print(model)

/home/qwj/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /data/MODELS/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)
